# Tagoloan Data Mining Notebook

<div>
    <img src='https://riverbasin.denr.gov.ph/img/Tagoloan%20RB/TagoloanRB3.jpg' width='500px'/>
</div>

This notebook processes the cleaned data from the data cleaning notebook. Processes of this notebook includes the following:
1. Removes hour and minute from the timestamp.
2. Clean duplicated data by keeping the first duplicated variable.
3. Ammend missing Water Level Data by Interpolation.
4. File integrity checker

### Authors
- Gifrey John M. Sulay

### Checked by:
- Dr. Anabel A. Abuzo
- Engr. Augustine Ave Padagunan

# CODE PROPER

### Import Modules and Foo Data

In [29]:
import numpy as np
import pandas as pd
import re
import datetime
# from first_gen_module import *

__author__ = "Gifrey John M. Sulay"
__copyright__ = "Xavier University - Engineering Resource Center"

### Open Dataset

In [30]:
#open data
while True:
    try:
        path_select = int(input('Select 1 for Tagoloan, 2 for San Isidro'))
        if path_select == 1:
            exec("""tag_path='Edited_Data/Tagoloan/tag.csv'
rg0_path='Edited_Data/Tagoloan/rg0.csv'
rg1_path='Edited_Data/Tagoloan/rg1.csv'
saved_data_path = 'Cleaned_Tagoloan'""")
        elif path_select == 2:
            exec("""tag_path='Edited_Data/Cagayan/tag.csv'
rg0_path='Edited_Data/Cagayan/rg0.csv'
rg1_path='Edited_Data/Cagayan/rg1.csv'
saved_data_path = 'Cleaned_Cagayan'""")
        else:
            raise Exception('Value not found. Please try again.')
            continue
        break
    except Exception as e:
        print(e)
tag=pd.read_csv(tag_path)
rg1=pd.read_csv(rg1_path)
rg0=pd.read_csv(rg0_path)
                
start="2021-12-01"
end="2022-01-01"

invalid literal for int() with base 10: ''
invalid literal for int() with base 10: ''
invalid literal for int() with base 10: ''
invalid literal for int() with base 10: ''


In [31]:
tag.head(3)

,Unnamed: 0,Station Name,Sensor Type,Sensor Value,Sensor ID,Units,Timestamp,Sensor Label
0,0,FGEN Tagoloan Gauge Station,Level,1.654815,46481,Meters,12/2/2019 12:47:00 AM,Water Level
1,1,FGEN Tagoloan Gauge Station,Level,1.678690,46481,Meters,12/2/2019 1:47:00 AM,Water Level
2,2,FGEN Tagoloan Gauge Station,Level,1.654815,46481,Meters,12/2/2019 2:47:00 AM,Water Level


In [32]:
rg0.head(3)

,Unnamed: 0,Station Name,Sensor Type,Sensor Value,Sensor ID,Units,Timestamp,Sensor Label
0,0,FGEN Tagoloan Gauge Station,Rain,207.2,46491,mm,1/27/2020 12:47:00 AM,Rain Gauge
1,1,FGEN Tagoloan Gauge Station,Rain,207.2,46491,mm,1/27/2020 1:47:00 AM,Rain Gauge
2,2,FGEN Tagoloan Gauge Station,Rain,207.2,46491,mm,1/27/2020 2:47:00 AM,Rain Gauge


In [33]:
rg1.head(3)

,Unnamed: 0,Station Name,Sensor Type,Sensor Value,Sensor ID,Units,Timestamp,Sensor Label
0,0,FGEN Tagoloan Gauge Station,DailyRain,0.112,1546480,mm,10/1/2020 12:59:59 AM,Rain Gauge 1
1,1,FGEN Tagoloan Gauge Station,DailyRain,0.112,1546480,mm,10/1/2020 1:59:59 AM,Rain Gauge 1
2,2,FGEN Tagoloan Gauge Station,DailyRain,0.112,1546480,mm,10/1/2020 2:59:59 AM,Rain Gauge 1


### Cleaning
This section does the following
1. Acquires *Timestamp* and *Sensor Value* data from cleaned data set.
2. Removes minute and seconds in *Timestamp* column.
3. Creates dataframe of duplicated values.

In [34]:
#filter data
tag_f=tag[['Timestamp','Sensor Value']].rename(columns={'Sensor Value':'Water_Level'})
rg0_f=rg0[['Timestamp','Sensor Value']].rename(columns={'Sensor Value':'RG0_Level'})
rg1_f=rg1[['Timestamp','Sensor Value']].rename(columns={'Sensor Value':'RG1_Level'})

In [35]:
def timestamp_conv(timestamp):
    timestamp=timestamp.split()
    date=timestamp[0]
    time=timestamp[1]
    hms=time.split(':')
    if len(timestamp)==3:
        morn_aft=timestamp[2]
        if int(hms[0])==12 and morn_aft=='AM':
            hour=0
        elif int(hms[0])==12 and morn_aft=='PM':
            hour=12
        elif morn_aft=='PM':
            hour=int(hms[0])+12
        elif morn_aft=='AM':
            hour=int(hms[0])

    if len(timestamp)==2:
        hour=int(hms[0])
    #reorganized timestamp to y/m/d
    date_split=date.split('/')
    month=date_split[0]
    day=date_split[1]
    year=date_split[2]
    reorganized_date=f"{int(year)}/{int(month)}/{int(day)}"
    new_timestamp=f"{reorganized_date} {int(hour)}:00:00"
    return new_timestamp

def timestamp_conv(timestamp):
    return pd.to_datetime(timestamp, infer_datetime_format=True).strftime('%Y/%m/%d %H:00:00')

In [36]:
#convert timestamps
tag_n = tag_f.copy()
tag_n['Timestamp']=tag_n['Timestamp'].apply(timestamp_conv)
rg0_n = rg0_f.copy()
rg0_n['Timestamp']=rg0_n['Timestamp'].apply(timestamp_conv)
rg1_n = rg1_f.copy()
rg1_n['Timestamp']=rg1_n['Timestamp'].apply(timestamp_conv)

In [37]:
#generate timestamp array for timestamp index for dataframe
#returns clean_timestamp
import datetime

clean_timestamp=pd.date_range(start=start, end=end, freq='H').strftime('%Y/%m/%d %H:00:00').tolist()

In [38]:
def duplicates_df(df):
    try:
        new_df = pd.concat([g for _, g in df.groupby("Timestamp") if len(g) > 1])
        return new_df
    except:
        new_df = df.iloc[0:0]
        return new_df
#create df of duplicated values
tag_n_duplicates=duplicates_df(tag_n)
rg0_n_duplicates=duplicates_df(rg0_n)
rg1_n_duplicates=duplicates_df(rg1_n)

#drop redundant data points
tag_n.drop_duplicates(keep='first', inplace=True)
rg0_n.drop_duplicates(keep='first', inplace=True)
rg1_n.drop_duplicates(keep='first', inplace=True)

### Merging
This section accomplishes the following:
1. Merges Water Level to Each Rain Gauge

In [39]:
#create dataframe for complete and correct timestamp
ts=pd.DataFrame({'Timestamp':clean_timestamp})

#create base dataframe(timestamp and water level)
ts_tag_n=pd.merge(ts,tag_n,how='left',on='Timestamp')
base=ts_tag_n

In [40]:
#merge base and rain gauge 0
base_rg0=pd.merge(base,rg0_n,how='left',on='Timestamp')
base_rg0

,Timestamp,Water_Level,RG0_Level
0,2021/12/01 00:00:00,0.413315,344.0
1,2021/12/01 01:00:00,0.914690,344.0
2,2021/12/01 02:00:00,0.484940,344.0
3,2021/12/01 03:00:00,0.986315,344.0
4,2021/12/01 04:00:00,0.580440,344.0
...,...,...,...
740,2021/12/31 20:00:00,1.320565,612.2
741,2021/12/31 21:00:00,1.439940,612.2
742,2021/12/31 22:00:00,0.914690,612.2
743,2021/12/31 23:00:00,1.439940,612.2


In [41]:
#merge base and rain gauge 1
base_rg1=pd.merge(base,rg1_n,how='left',on='Timestamp')
base_rg1

,Timestamp,Water_Level,RG1_Level
0,2021/12/01 00:00:00,0.413315,0.088
1,2021/12/01 01:00:00,0.914690,0.088
2,2021/12/01 02:00:00,0.484940,0.088
3,2021/12/01 03:00:00,0.986315,0.088
4,2021/12/01 04:00:00,0.580440,0.088
...,...,...,...
740,2021/12/31 20:00:00,1.320565,0.056
741,2021/12/31 21:00:00,1.439940,0.056
742,2021/12/31 22:00:00,0.914690,0.056
743,2021/12/31 23:00:00,1.439940,0.056


In [57]:
base_rg0.loc[pd.isna(base_rg1['Water_Level'])]
#base_rg1.loc[]

,Timestamp,Water_Level,RG0_Level
304,2021/12/13 16:00:00,NaN,NaN
306,2021/12/13 18:00:00,NaN,NaN


### First Save
This section does the following:
1. Saves merged datas of each rain gauge to **Cleaned_Data.xlsx**
2. Saves dataframe of missing values to **Missing Values.xlsx**

In [43]:
# #save to excel sheet
# with pd.ExcelWriter(f'{saved_data_path}/Cleaned_Data.xlsx') as writer:
#     base_rg0.to_excel(writer, sheet_name='Water Level - Rain Gauge 0')
#     base_rg1.to_excel(writer, sheet_name='Water Level - Rain Gauge 1')

In [44]:
#create spreadsheet of missing values
water_level_missing=base_rg0[base_rg0['Water_Level'].isnull()].index.tolist()
rg0_missing=base_rg0[base_rg0['RG0_Level'].isnull()].index.tolist()
rg1_missing=base_rg1[base_rg1['RG1_Level'].isnull()].index.tolist()

water_level_missing_df=pd.DataFrame({'Missing Water Level':water_level_missing})
rg0_missing_df=pd.DataFrame({'Missing RG0':rg0_missing})
rg1_missing_df=pd.DataFrame({'Missing RG1':rg1_missing})

with pd.ExcelWriter(f'{saved_data_path}/Missing Values.xlsx') as writer:
    water_level_missing_df.to_excel(writer, sheet_name='Water Level')
    rg0_missing_df.to_excel(writer, sheet_name='Rain Gauge 0')
    rg1_missing_df.to_excel(writer, sheet_name='Rain Gauge 1')
    
with pd.ExcelWriter(f'{saved_data_path}/Duplicated Values.xlsx') as writer:
    tag_n_duplicates.set_index('Timestamp').to_excel(writer, sheet_name='Water Level')
    rg0_n_duplicates.set_index('Timestamp').to_excel(writer, sheet_name='Rain Gauge 0')
    rg1_n_duplicates.set_index('Timestamp').to_excel(writer, sheet_name='Rain Gauge 1')

### Ammendment of Missing Values
This section does the following
1. Water Level
    - Interpolate missing values using np.interpolate's linear method.
2. Rain Gauge 0
    - Replace *None* values in Rain Gauge 0 using np.interpolate's padding method.
3. Saves the ammended data in **Corrected_Data.xlsx**.

**Note:**
Data given in Rain Gauge 1 is unclear whether the data given is correct. *Thus, Rain Gauge 1 will not be used in the succeeding analysis.*

### Rain Gauge 0

In [45]:
#interpolate water level
new_water_level=base['Water_Level'].astype('float64').interpolate()

#replace nan values in RG0 using np.interpolate's padding method
rg0_copy=base_rg0['RG0_Level'].astype('float64').interpolate()

#create difference on Rain Gauge 0
rg0_diff=[0]
count=1
for i in rg0_copy[1:]:
    diff=i-rg0_copy[count-1]
    if diff < 0:
        diff = 0
    rg0_diff.append(diff)
    count+=1
    
rg0_copy=base_rg0['RG0_Level'].astype('float64').interpolate()

#create dataframe with corrected data using base dataframe
corrected_df=base
corrected_df['Corrected_Water_Level']= new_water_level
corrected_df=corrected_df.drop(columns=['Water_Level'])
corrected_df['Corrected_RG0_Level']=rg0_copy
corrected_df['RG0_Diff']=rg0_diff
corrected_df=corrected_df.reset_index(drop = True)

def hours_only(df):
    timestamp_arr=df['Timestamp']
    hour_lst=[]
    for i in timestamp_arr:
        time=i.split()[1]
        hour=time.split(':')[0]
        hour_lst.append(hour)
    df['Hour']=hour_lst
    return df

corrected_df=hours_only(corrected_df)
headers=corrected_df.columns.values.tolist()
headers.remove('Hour')
headers.insert(0,'Hour')
corrected_df=corrected_df[headers]
corrected_df0 = corrected_df
corrected_df0

,Hour,Timestamp,Corrected_Water_Level,Corrected_RG0_Level,RG0_Diff
0,00,2021/12/01 00:00:00,0.413315,344.0,0.0
1,01,2021/12/01 01:00:00,0.914690,344.0,0.0
2,02,2021/12/01 02:00:00,0.484940,344.0,0.0
3,03,2021/12/01 03:00:00,0.986315,344.0,0.0
4,04,2021/12/01 04:00:00,0.580440,344.0,0.0
...,...,...,...,...,...
740,20,2021/12/31 20:00:00,1.320565,612.2,0.0
741,21,2021/12/31 21:00:00,1.439940,612.2,0.0
742,22,2021/12/31 22:00:00,0.914690,612.2,0.0
743,23,2021/12/31 23:00:00,1.439940,612.2,0.0


In [46]:
base_rg1.loc[base_rg1['Water_Level'].isnull()].fillna(1).count()

Timestamp      2
Water_Level    2
RG1_Level      2
dtype: int64

### Rain Gauge 1

In [47]:
#interpolate water level
new_water_level=base['Water_Level'].astype('float64').interpolate()

#replace nan values in RG1 using np.interpolate's padding method
rg1_copy=base_rg1['RG1_Level'].astype('float64').interpolate()

#create difference on Rain Gauge 0
rg1_diff=[0]
count=1
for i in rg1_copy[1:]:
    diff=i-rg1_copy[count-1]
    if diff < 0:
        diff = 0
    rg1_diff.append(diff)
    count+=1
rg1_copy=base_rg1['RG1_Level'].astype('float64').interpolate()

#create dataframe with corrected data using base dataframe
corrected_df=base
corrected_df['Corrected_Water_Level']= new_water_level
corrected_df=corrected_df.drop(columns=['Water_Level'])
corrected_df['Corrected_RG1_Level']=rg1_copy
corrected_df['RG1_Diff']=rg1_diff
corrected_df=corrected_df.reset_index(drop = True)

def hours_only(df):
    timestamp_arr=df['Timestamp']
    hour_lst=[]
    for i in timestamp_arr:
        time=i.split()[1]
        hour=time.split(':')[0]
        hour_lst.append(hour)
    df['Hour']=hour_lst
    return df

corrected_df=hours_only(corrected_df)
headers=corrected_df.columns.values.tolist()
headers.remove('Hour')
headers.insert(0,'Hour')
corrected_df=corrected_df[headers]
corrected_df1 = corrected_df
corrected_df1

,Hour,Timestamp,Corrected_Water_Level,Corrected_RG1_Level,RG1_Diff
0,00,2021/12/01 00:00:00,0.413315,0.088,0.000
1,01,2021/12/01 01:00:00,0.914690,0.088,0.000
2,02,2021/12/01 02:00:00,0.484940,0.088,0.000
3,03,2021/12/01 03:00:00,0.986315,0.088,0.000
4,04,2021/12/01 04:00:00,0.580440,0.088,0.000
...,...,...,...,...,...
740,20,2021/12/31 20:00:00,1.320565,0.056,0.000
741,21,2021/12/31 21:00:00,1.439940,0.056,0.000
742,22,2021/12/31 22:00:00,0.914690,0.056,0.000
743,23,2021/12/31 23:00:00,1.439940,0.056,0.000


In [48]:
with pd.ExcelWriter(f'{saved_data_path}/Corrected_Data.xlsx') as writer:
    corrected_df0.to_excel(writer,sheet_name='Corrected_Water_and_RG0_Level')
    corrected_df1.to_excel(writer,sheet_name='Corrected_Water_and_RG1_Level')

### Dataframe Integrity Check
Checks if dataframe has missing values

In [49]:
def check_integrity(df):
    return df[df.isnull().any(axis=1)]
x=check_integrity(corrected_df)
x

,Hour,Timestamp,Corrected_Water_Level,Corrected_RG1_Level,RG1_Diff


### Ammendment 1: Erroneous Water Level
Erronoeous water level data were found. Data are catergorized as erroneous if the difference between the previous and current datapoint exceeds the stated difference limite(diff_limit)

Objective:
* Create array and collect velues larger than stated difference limit
* Remove erroneous data
* Interpolate removed values


In [50]:
def clean_water_level(corrected_df, diff_limit = 0.5):
    water_level = corrected_df['Corrected_Water_Level']
    #diff limit determines whether the difference is erroneous if difference is more than diff limit then data is erroneous
    is_erroneous=[0]
    count = 1
    
    for i in water_level[1:]:
        val = i-water_level[count - 1]
        val = abs(val)
        if val >= diff_limit:
            is_erroneous.append(1)
        else:
            is_erroneous.append(0)
        count += 1
    corrected_df['err_check']=is_erroneous
    corrected_df.loc[corrected_df['err_check']==1, 'Corrected_Water_Level'] = np.nan
    edited_water_level=corrected_df.loc[:, 'Corrected_Water_Level']
    final_water_level=edited_water_level.astype('float64').interpolate()
    corrected_df=corrected_df.drop(columns=['Corrected_Water_Level','err_check'])
    corrected_df['Corrected_Water_Level']=final_water_level
    print(corrected_df)
    return corrected_df

In [51]:
corrected_df0 = clean_water_level(corrected_df0)
corrected_df1 = clean_water_level(corrected_df1)

    Hour            Timestamp  Corrected_RG0_Level  RG0_Diff  \
0     00  2021/12/01 00:00:00                344.0       0.0   
1     01  2021/12/01 01:00:00                344.0       0.0   
2     02  2021/12/01 02:00:00                344.0       0.0   
3     03  2021/12/01 03:00:00                344.0       0.0   
4     04  2021/12/01 04:00:00                344.0       0.0   
..   ...                  ...                  ...       ...   
740   20  2021/12/31 20:00:00                612.2       0.0   
741   21  2021/12/31 21:00:00                612.2       0.0   
742   22  2021/12/31 22:00:00                612.2       0.0   
743   23  2021/12/31 23:00:00                612.2       0.0   
744   00  2022/01/01 00:00:00                612.4       0.2   

     Corrected_Water_Level  
0                 0.413315  
1                 0.449128  
2                 0.484940  
3                 0.532690  
4                 0.580440  
..                     ...  
740               1.320565  

In [52]:
with pd.ExcelWriter(f'{saved_data_path}/Corrected_Data.xlsx') as writer:
    corrected_df0.to_excel(writer,sheet_name='Corrected_Water_and_RG0_Level')
    corrected_df1.to_excel(writer,sheet_name='Corrected_Water_and_RG1_Level')

### Ammendment 2: Daily Water Count
Create daily where per day max water level and cumulated rain gauge data is added to new data frame

In [53]:
def daily_water_data(df):
    daily_df_timestamp = []
    count = 1
    for i in df['Timestamp']:
        if count%24 != 0:
            count += 1
            continue
        else:
            date=i.split()[0]
            daily_df_timestamp.append(date)
        count += 1
        
    daily_df_rain_gauge = []
    count=1
    temp = []
    for i in df.RG0_Diff:
        if count % 24 != 0:
            temp.append(i)
            count += 1
        else:
            total = 0
            for n in temp:
                total = total + n
            daily_df_rain_gauge.append(total)
            temp = []
            temp.append(i)
            count += 1
            
    daily_water_level = []
    count=1
    temp = []

    for i in df.Corrected_Water_Level:
        if count % 24 != 0:
            temp.append(i)
            count += 1
        else:
            total = 0
            for n in temp:
                if total < n:
                    total = n
                else:
                    continue
            daily_water_level.append(total)
            temp = []
            temp.append(i)
            count += 1
    daily_df = pd.DataFrame({'Timestamp':daily_df_timestamp,'Water_Level':daily_water_level,'Rain Gauge':daily_df_rain_gauge})
    daily_df=daily_df.set_index('Timestamp')
    return daily_df

In [54]:
daily_df = daily_water_data(corrected_df0)

In [55]:
daily_df

,Water_Level,Rain Gauge
Timestamp,,
2021/12/01,1.034065,0.0
2021/12/02,1.010190,0.0
2021/12/03,0.962440,2.8
2021/12/04,0.986315,0.0
2021/12/05,0.962440,3.4
2021/12/06,0.962440,0.0
2021/12/07,0.938565,5.0
2021/12/08,0.962440,0.0
2021/12/09,1.034065,21.8


In [56]:
with pd.ExcelWriter(f'{saved_data_path}/Daily_Data.xlsx') as writer:
    daily_df.to_excel(writer,sheet_name='Sheet 1')